1. First create <b>`Dockerfile`</b> file

In [ ]:
                                                                                         # Dockerfile
# Bring Base Image From The Docker Hub
FROM python:3.7-alpine

# Name Of The Code Maintainer & Email 
MAINTAINER Saifullah al Mujahid <mujahid7292@gmail.com>

# Let set python unbuffered environment variable.
ENV PYTHONUNBUFFERED 1
# What this environment variable does, is that it tell python
# to run in unbuffered mode. This is recommended when running 
# python inside docker container. The reason for this, is that 
# it does not allow python to buffer the output. So it reduce
# the error in running python inside docker image.

# We are going to store our projects dependencies in the
# ./requirements.txt file.
COPY ./requirements.txt  /requirements.txt
# This command will copy this file to inside the docker image
# in the following '/' directory.

# Now we will install all of the dependencies of our project.
RUN pip install -r /requirements.txt

# We will create a directory(/app) within our docker image, that we can use
# to store our application source code.
RUN mkdir /app
# Create a directory/folder named 'app' inside our docker image
WORKDIR /app
# Switched to this 'app' directory as a default directory.
# Any application that we run inside our docker container, will start
# running from this location.
COPY ./app /app
# Copy all the source code from our local machine / 'app' folder to the directory
# inside docker container which is also 'app' folder.

# We will create an user who will run our application using docker.
RUN adduser -D mujahid7292
# -D = Create an user without admin privilage. That means this user will only be able 
# to run this docker image & will not get any root access.
# mujahid7292 = Name of the user.
USER mujahid7292
# Now docker will switched to this 'mujahid7292' user.


2. Create <b>`requirements.txt`</b> file

In [ ]:
                                                                                     #requirements.txt
Django>=2.1.4,<2.2.0
djangorestframework>=3.9.0,<3.10.0

3. Create <b>`docker-compose.yml`</b> file

In [ ]:
                                                                                # docker-compose.yml
version: "3"
# docker-compose version number. 3 is the lattest version

# Now we will define services that make up our application
services:
  app: # This is the name of our services
    build:
      context: . # This will set the build to our current directory
    ports:
      - "8000:8000"
      # First 8000, means we are exposing port 8000 in our machine.
      # Second 8000, means we are exposing port 8000 in our docker image.
    volumes:
    # Volumes allows us to get the update (that we make to our project) into our docker
    # image in real time. So this section will maps a volume from our local machine
    # to our docker container. This means whenever you change some code or file in
    # the project, it will automatically get updated into the container. That means
    # you don't need to restart the docker to get the changes.
      - ./app:/app
    # ./app = 'app' directory in our local machine
    # /app =  'app' directory inside the docker image

    # Now we will use 'command' to run our application inside docker container.
    command: > # > = This symbol break the command to next line
      sh -c "python manage.py runserver 0.0.0.0:8000"
      # sh = We are going to run the command using shell
      # -c = Run following command
      

4. In the directory where docker-compose.yml file has, run following command in the windows powershell or command line
<br/><b>`$ docker-compose build`</b>
<br/>This command will build the image & give it the name of the folder `restapiwithdjangoudemy_app:latest`

5. Now we will create a django project, using the above docker configuration. Now create our django project by this command:
<br/><b>`$ sudo docker-compose run app sh -c "django-admin.py startproject app ."`</b>
<br/> app = Name of the service which you want to run. Currently we have only one service. Anything after this word will pass into this linux container.
<br/> sh  = Run a shell script inside linux container.
<br/> -c = This is the flag for passing command into linux container.
<br/> "" = Inside this we pass our command.
<br/> django-admin.py startproject app = This will create a new django project name `app`.
<br/> . = The project will be created in the current directory.

# Section-5:

### Video: 20. Create Travis-CI configuration file

The travis ci configuration file is the file, that tells travis ci what to do, everytime we push a change to our project.

You create a travis ci configuration file by simply creating file in the root of the project directory. You name the file as <b>`.travis.yml`</b>

In [ ]:
                                                                                          #.travis.yml
#In the first line we will tell travis CI what language we have used in this
#project. As we have created this project using python. So we will write.
language: python

# Next we will tell travis CI what version of python it should use to spin
# up the server to run automated test.
python:
 - "3.6"

#Next we are going to tell travis CI, what service we need to use.
services:
 - "docker"
# We are just going to need docker service for this project. And all of
#the sub services are gonna be contained within our docker-compose.yml file
#and our Dockerfile configuration.

#Now we can specify a before_script, which is a script that travis will run
#before it executes any of the automation command, that we are going to input
#after before_script.
before_script: pip install docker-compose
#Before it runs any thing we need to install docker compose.

#Next we specify the script.
script:
  #Here we are going to run our docker-compose command for running
  #our automated test.
  - docker-compose run app sh -c "python manage.py test && flake8"
  #We also will run our linting tool by adding "flake8" with the
  #above command.

Next we need to add this (flake8) linting tool in our project. For that we will add <b>`flake8>=3.6.0,<3.7.0`</b> in our requirement.txt file.

In [1]:
                                                                                     #requirements.txt
Django>=2.1.4,<2.2.0
djangorestframework>=3.9.0,<3.10.0
flake8>=3.6.0,<3.7.0

SyntaxError: invalid syntax (<ipython-input-1-e604a771ade5>, line 2)

Next we will add flake8 config file inside our django project. Name of the file will be <b>`.flake8`</b>

In [ ]:
                                                                                             #.flake8
[flake8]
exclude=
 migrations,
 __pycahce__,
 manage.py,
 settings.py

Afte adding `.flake8` config file in our project, lets rebuild our docker image by below command:
<br/><b>`$ sudo docker-compose build`</b>

Next commit & push our changes to the github.

# Section-6:

### Video: 21. Writing a simple unit test

Lets create a new file in our app directory name calc.py

In [ ]:
                                                                                            #calc.py
def add(x,y):
    """
    This function add two number together.
    """
    return x + y

Now lets say we want create an unit test for our add function. To do this we are going to create a new file name <b>`tests.py`</b> in the same directory.

Django unit test framework looks for any file /folder that begins with the word `test` and it basically use those file / folder as a test script, when you run django `run unit test` command. 

In [ ]:
                                                                                            #tests.py
from django.test import TestCase
# This above import will help us to test our django app.

#Next we will import function which we want to test.
from app.calc import add

#Now we will create a classs which will test our calc
#class.
class CalcTest(TestCase):
    #We are going to inherit this class from TestCase

    # Now we will create test function to test the add
    #function.

    def test_add_numbers(self):
        """
        Test that two numbers are added together.
        """
        self.assertEqual(add(3,8),11)

One thing to mention about this function name <b>`test_add_numbers`</b> is that, just like when django search for file / folder with the start name `test`, this test function should also start with the name `test`.

Now lets save this <b>`tests.py`</b> file & go to our terminal and run django unit test by this below command:
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test"`</b>

### Video: 22. Writing a unit test with TDD

We will also check python linting, So:
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test && flake8"`</b>

# Section 7: Configure Django custom user model

### Video: 23. Create core app

The first thing we're going to do is we're going to create a core app which will hold all of the central code that is important to the rest of the sub apps that we create in our system. So it's going to create anything that is shared between one or more apps. So things like the migration and the database I like to put this all in the core module just so it's all in one place and is very clear where the kind of central point of all these things is.

All right we're going to create our app from the terminal and we're going to use the django manage
command just like we did to create the project. So we're going to type:

<br/><b>`$ sudo docker-compose run app sh -c "python manage.py startapp core"`</b>

Let's clean up some of the files that we're not going to be using in this `core` folder. So we're not going to be using the `tests.py` file, So delete this file And let's remove the `views.py` because we're not going to need the views in the core. 

And then let's create a new folder called `tests` and inside this folder Let's create `__init__.py`.

### Video: 24. Add tests for custom user

First Let's go into the settings of our app and make sure that the core app that we created in the previous video is added to the installed apps list.

In [ ]:
#settings.py

INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'core', # Here we have added our core app.
]

Once as added We can go into our `tests` folder and we can create a new file called `test_models.py`

In [ ]:
                                                                                      #test_models.py
from django.test import TestCase
from django.contrib.auth import get_user_model

class ModelTests(TestCase):

    def test_create_user_with_email_successfull(self):
        """
        Test that will test whether creating a new user with
        email and password is suceessful.
        """

        # Now create one fake email & password for testing the
        # user object
        email = "test_email@gmail.com"
        password = "TestPassword123"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email
            password = password
        )

        # Test the fake user object, whether properly created
        # or not.
        self.assertEqual(user.email,email)
        self.assertTrue(user.check_password(password))

Now, we will run our test:
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test && flake8"`</b>

### Video: 25. Implement custom user model

The first thing we're going to do is we have to create a user model in our `models.py` file in the
core app and then we're going to update the `settings.py` file to set our custom auth user model.

In [1]:
                                                                                           #models.py
from django.db import models
from django.contrib.auth.models import AbstractBaseUser, \
    BaseUserManager, PermissionMixin

# Create a model manager class. As our model name is 'user',
# So, model manager class name will be 'UserModel'
class UserManager(BaseUserManager):

    def create_user(self, email, password=None, **extra_fields):
        """
        This function creates and saves a new user.

        email = User email address.
        password = User provided password. Default is none.
        **extra_fields = This will allow to add new field when
                    we create user. Such as user phone number
                    etc.

        return
        This function return a newly created user.
        """

        user = self.model(email=email,**extra_fields)
        user.set_password(password)
        user.save(using=self._db)

        # 'using=self._db' = This will help us to support
        # multiple database.

        return user

# Now we will create our model class

class User(AbstractBaseUser, PermissionMixin):
    """
    This is custom user model, that supports email base
    user authentication rather than name base user 
    authentication.
    """

    email = models.EmailField(max_length=255, unique=True)
    name = models.CharField(max_length=255)
    is_active = models.BooleanField(default=True)
    is_staff = models.BooleanField(default=False)

    objects = UserManager()

    USERNAME_FIELD = 'email'
    # In django by default USERNAME_FIELD = 'username', but
    # in this app we are stting it to be email.

In [ ]:
                                                                                         #settings.py

########################### At the bottom of the file add this.###################################

# Settings added by me
AUTH_USER_MODEL = 'core.User'
# Here 'core' is the name of the app
# And 'User' is the name of our model

Now go back to our terminal and make the migration.
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py makemigrations core"`</b>

Here,
core = It is the name of the app for which we are running this above migration command.

Now that we have complete our database migration successfully, we can run our test again.
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test && flake8"`</b>

### Video: 26. Normalize email addresses

In [ ]:
                                                                                       #test_models.py
from django.test import TestCase
from django.contrib.auth import get_user_model

class ModelTests(TestCase):

    def test_create_user_with_email_successfull(self):
        """
        Test that will test whether creating a new user with
        email and password is suceessful.
        """

        # Now create one fake email & password for testing the
        # user object
        email = "test_email@gmail.com"
        password = "TestPassword123"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email,
            password = password
        )

        # Test the fake user object, whether properly created
        # or not.
        self.assertEqual(user.email,email)
        self.assertTrue(user.check_password(password))


    def test_new_user_email_normalized(self):
        """
        This test will check whether we have implemented
        email text normalized / convert all upper case letter
        in email to lowercase or not.
        """

        # Now we will create a dummy email with all upper case
        # after @ sign 
        email = "test_email@GMAIL.COM"
        pass = "Test1234"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email,
            password = pass
        )

        # Now we will check whether our upper case email
        # has been converted into lower case.
        self.assertEqual(user.email, email.lower())


Now go back to our terminal & run only python test this time not flake8 test.
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test"`</b>

And we will see that our test failed. So to pass the test we will modify our `models.py` file.

In [ ]:
from django.db import models
from django.contrib.auth.models import AbstractBaseUser, \
    BaseUserManager, PermissionsMixin

# Create a model manager class. As our model name is 'user',
# So, model manager class name will be 'UserModel'
class UserManager(BaseUserManager):

    def create_user(self, email, password=None, **extra_fields):
        """
        This function creates and saves a new user.

        email = User email address.
        password = User provided password. Default is none.
        **extra_fields = This will allow to add new field when
                    we create user. Such as user phone number
                    etc.

        return
        This function return a newly created user.
        """

        user = self.model(
            email=self.normalize_email(email),
            **extra_fields
            )
        user.set_password(password)
        user.save(using=self._db)

        # 'using=self._db' = This will help us to support
        # multiple database.

        return user

# Now we will create our model class

class User(AbstractBaseUser, PermissionsMixin):
    """
    This is custom user model, that supports email base
    user authentication rather than name base user 
    authentication.
    """

    email = models.EmailField(max_length=255, unique=True)
    name = models.CharField(max_length=255)
    is_active = models.BooleanField(default=True)
    is_staff = models.BooleanField(default=False)

    objects = UserManager()

    USERNAME_FIELD = 'email'
    # In django by default USERNAME_FIELD = 'username', but
    # in this app we are stting it to be email.

### Video: 27. Add validation for email field

Next we're going to add validation to ensure that an e-mail field has actually been provided when create_user() function is called. So lets write new test in our `test_models.py` file.

In [ ]:
                                                                                    #test_models.py
from django.test import TestCase
from django.contrib.auth import get_user_model

class ModelTests(TestCase):

    def test_create_user_with_email_successfull(self):
        """
        Test that will test whether creating a new user with
        email and password is suceessful.
        """

        # Now create one fake email & password for testing the
        # user object
        email = "test_email@gmail.com"
        password = "TestPassword123"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email,
            password = password
        )

        # Test the fake user object, whether properly created
        # or not.
        self.assertEqual(user.email,email)
        self.assertTrue(user.check_password(password))


    def test_new_user_email_normalized(self):
        """
        This test will check whether we have implemented
        email text normalized / convert all upper case letter
        in email to lowercase or not.
        """

        # Now we will create a dummy email with all upper case
        # after @ sign 
        email = "test_email@GMAIL.COM"
        password = "Test1234"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email,
            password = password
        )

        # Now we will check whether our upper case email
        # has been converted into lower case.
        self.assertEqual(user.email, email.lower())


    def test_invalid_user_email_raise_value_error(self):
        """
        This test will ensure that if we try to create a new
        user without providing an emaill address, our function
        will raise a value error.
        """
        with self.assertRaises(ValueError):
            # anything that we run in here should 
            # raise the value error And if it doesn't 
            # raise the value error, then this test will fail.
            get_user_model().objects.create_user(None,'test123')

Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test"`</b>

And we will see that our test failed. So to pass the test we will modify our `models.py` file.

In [ ]:
                                                                                           #models.py
from django.db import models
from django.contrib.auth.models import AbstractBaseUser, \
    BaseUserManager, PermissionsMixin

# Create a model manager class. As our model name is 'user',
# So, model manager class name will be 'UserModel'
class UserManager(BaseUserManager):

    def create_user(self, email, password=None, **extra_fields):
        """
        This function creates and saves a new user.

        email = User email address.
        password = User provided password. Default is none.
        **extra_fields = This will allow to add new field when
                    we create user. Such as user phone number
                    etc.

        return
        This function return a newly created user.
        """
        if not email:
            raise ValueError('User must provide email address')

        user = self.model(
            email=self.normalize_email(email),
            **extra_fields
            )
        user.set_password(password)
        user.save(using=self._db)

        # 'using=self._db' = This will help us to support
        # multiple database.

        return user

# Now we will create our model class

class User(AbstractBaseUser, PermissionsMixin):
    """
    This is custom user model, that supports email base
    user authentication rather than name base user 
    authentication.
    """

    email = models.EmailField(max_length=255, unique=True)
    name = models.CharField(max_length=255)
    is_active = models.BooleanField(default=True)
    is_staff = models.BooleanField(default=False)

    objects = UserManager()

    USERNAME_FIELD = 'email'
    # In django by default USERNAME_FIELD = 'username', but
    # in this app we are stting it to be email.

### Video: 28. Add support for creating superusers

create_super_user() is a function used by the Django command line interface (cli), when we are creating new users using the command line.

In [ ]:
                                                                                     #test_models.py
from django.test import TestCase
from django.contrib.auth import get_user_model

class ModelTests(TestCase):

    def test_create_user_with_email_successfull(self):
        """
        Test that will test whether creating a new user with
        email and password is suceessful.
        """

        # Now create one fake email & password for testing the
        # user object
        email = "test_email@gmail.com"
        password = "TestPassword123"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email,
            password = password
        )

        # Test the fake user object, whether properly created
        # or not.
        self.assertEqual(user.email,email)
        self.assertTrue(user.check_password(password))


    def test_new_user_email_normalized(self):
        """
        This test will check whether we have implemented
        email text normalized / convert all upper case letter
        in email to lowercase or not.
        """

        # Now we will create a dummy email with all upper case
        # after @ sign 
        email = "test_email@GMAIL.COM"
        password = "Test1234"

        # Create fake user object using above emaill & pass
        user = get_user_model().objects.create_user(
            email = email,
            password = password
        )

        # Now we will check whether our upper case email
        # has been converted into lower case.
        self.assertEqual(user.email, email.lower())


    def test_invalid_user_email_raise_value_error(self):
        """
        This test will ensure that if we try to create a new
        user without providing an emaill address, our function
        will raise a value error.
        """
        with self.assertRaises(ValueError):
            # anything that we run in here should 
            # raise the value error And if it doesn't 
            # raise the value error, then this test will fail.
            get_user_model().objects.create_user(None,'test123')

    def test_create_new_super_user(self):
        """
        This test will check whether a new user is 
        created and have the super user & staf tag 
        assigned to it.
        """
        user = get_user_model().objects.create_superuser(
            'test_123@gmail.com',
            'Test123'
        )

        # check whether new user is super user
        self.assertTrue(user.is_superuser)
        # check whether new user is staff
        self.assertTrue(user.is_staff)

Now go back to our terminal & run the test .
<br/><b>`$ sudo docker-compose run app sh -c "python manage.py test"`</b>

And we will see that our test failed. So to pass the test we will modify our `models.py` file.

In [ ]:
                                                                                        #models.py
from django.db import models
from django.contrib.auth.models import AbstractBaseUser, \
    BaseUserManager, PermissionsMixin

# Create a model manager class. As our model name is 'user',
# So, model manager class name will be 'UserModel'
class UserManager(BaseUserManager):

    def create_user(self, email, password=None, **extra_fields):
        """
        This function creates and saves a new user.

        email = User email address.
        password = User provided password. Default is none.
        **extra_fields = This will allow to add new field when
                    we create user. Such as user phone number
                    etc.

        return
        This function return a newly created user.
        """
        if not email:
            raise ValueError('User must provide email address')

        user = self.model(
            email=self.normalize_email(email),
            **extra_fields
            )
        user.set_password(password)
        user.save(using=self._db)

        # 'using=self._db' = This will help us to support
        # multiple database.

        return user

    
    def create_superuser(self, email, password):
        """
        Create a new super user & save it & return it
        """
        user = self.create_user(email, password)
        user.is_staff = True
        user.is_superuser = True
        user.save(using=self._db)

        return user

# Now we will create our model class

class User(AbstractBaseUser, PermissionsMixin):
    """
    This is custom user model, that supports email base
    user authentication rather than name base user 
    authentication.
    """

    email = models.EmailField(max_length=255, unique=True)
    name = models.CharField(max_length=255)
    is_active = models.BooleanField(default=True)
    is_staff = models.BooleanField(default=False)

    objects = UserManager()

    USERNAME_FIELD = 'email'
    # In django by default USERNAME_FIELD = 'username', but
    # in this app we are stting it to be email.

### Video: 29. Add tests for listing users in Django admin

In [ ]:
### Video: 25. Implement custom user model<b>``</b>
<b>`$ `</b>
<br/><b>`$ `</b>